Model generates poems.

Dataset source: https://www.kaggle.com/datasets/ultrajack/modern-renaissance-poetry

# Imports

In [ ]:
import os
import re
import itertools
import pandas as pd
import tensorflow as tf
import numpy as np
import gzip
import pickle

from google.colab import drive
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, TimeDistributed, Dense
from tensorflow.keras.callbacks import Callback
from keras.models import load_model

# Constants

In [ ]:
poem_generator_directory = '/content/gdrive/MyDrive/poem_generator'
stateful_rnn_file_name = 'stateful_model.h5'
stateless_rnn_file_name = 'stateless_model.h5'
dataset_file_name = 'all.csv'
tokenizer_file_name = 'tokenizer.pkl'

n_steps = 100
window_length = n_steps + 1

# Preparing the data

Mounting with Google Drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Reading, analyzing and preparing the dataset

In [ ]:
data = pd.read_csv(os.path.join(poem_generator_directory, dataset_file_name), header=0)
print(data.shape)
data.head()

(573, 5)


,author,content,poem name,age,type
0,WILLIAM SHAKESPEARE,Let the bird of loudest lay\r\nOn the sole Ara...,The Phoenix and the Turtle,Renaissance,Mythology & Folklore
1,DUCHESS OF NEWCASTLE MARGARET CAVENDISH,"Sir Charles into my chamber coming in,\r\nWhen...",An Epilogue to the Above,Renaissance,Mythology & Folklore
2,THOMAS BASTARD,"Our vice runs beyond all that old men saw,\r\n...","Book 7, Epigram 42",Renaissance,Mythology & Folklore
3,EDMUND SPENSER,"Lo I the man, whose Muse whilome did maske,\r\...","from The Faerie Queene: Book I, Canto I",Renaissance,Mythology & Folklore
4,RICHARD BARNFIELD,"Long have I longd to see my love againe,\r\nSt...",Sonnet 16,Renaissance,Mythology & Folklore


In [ ]:
data.describe()

,author,content,poem name,age,type
count,573,573,571,573,573
unique,67,506,508,2,3
top,WILLIAM SHAKESPEARE,"Originally published in Poetry, March 1914.",Canto IV,Renaissance,Love
freq,71,4,3,315,326


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author     573 non-null    object
 1   content    573 non-null    object
 2   poem name  571 non-null    object
 3   age        573 non-null    object
 4   type       573 non-null    object
dtypes: object(5)
memory usage: 22.5+ KB


In [ ]:
data.drop_duplicates()
print(data.shape)

(573, 5)


In [ ]:
data.isna().sum()

author       0
content      0
poem name    2
age          0
type         0
dtype: int64

In [ ]:
data = data['content']
data.head()

0    Let the bird of loudest lay\r\nOn the sole Ara...
1    Sir Charles into my chamber coming in,\r\nWhen...
2    Our vice runs beyond all that old men saw,\r\n...
3    Lo I the man, whose Muse whilome did maske,\r\...
4    Long have I longd to see my love againe,\r\nSt...
Name: content, dtype: object

In [ ]:
data = data.map(lambda text : re.sub('\W', ' ', text))

In [ ]:
data.head()

0    Let the bird of loudest lay  On the sole Arabi...
1    Sir Charles into my chamber coming in   When I...
2    Our vice runs beyond all that old men saw   An...
3    Lo I the man  whose Muse whilome did maske   A...
4    Long have I longd to see my love againe   Stil...
Name: content, dtype: object

In [ ]:
tokenizer = Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts(data)

In [ ]:
max_id = len(tokenizer.word_index)
data_size = tokenizer.document_count
print(max_id)
print(data_size)

64
573


In [ ]:
data_encoded = np.asarray(tokenizer.texts_to_sequences(data), dtype="object")

In [ ]:
data_encoded

array([list([32, 2, 3, 1, 3, 6, 2, 1, 20, 10, 9, 12, 1, 4, 18, 1, 11, 4, 13, 12, 2, 7, 3, 1, 11, 5, 16, 1, 1, 30, 8, 1, 3, 6, 2, 1, 7, 4, 11, 2, 1, 25, 9, 5, 20, 10, 5, 8, 1, 3, 9, 2, 2, 1, 1, 31, 2, 9, 5, 11, 12, 1, 7, 5, 12, 1, 5, 8, 12, 1, 3, 9, 13, 14, 21, 2, 3, 1, 20, 2, 1, 1, 1, 24, 4, 1, 15, 6, 4, 7, 2, 1, 7, 4, 13, 8, 12, 1, 19, 6, 5, 7, 3, 2, 1, 15, 10, 8, 17, 7, 1, 4, 20, 2, 16, 1, 1, 1, 1, 1, 29, 13, 3, 1, 3, 6, 4, 13, 1, 7, 6, 9, 10, 2, 23, 10, 8, 17, 1, 6, 5, 9, 20, 10, 8, 17, 2, 9, 1, 1, 1, 34, 4, 13, 11, 1, 21, 9, 2, 19, 13, 9, 9, 2, 9, 1, 4, 18, 1, 3, 6, 2, 1, 18, 10, 2, 8, 12, 1, 1, 1, 25, 13, 17, 13, 9, 1, 4, 18, 1, 3, 6, 2, 1, 18, 2, 22, 2, 9, 1, 7, 1, 2, 8, 12, 1, 1, 1, 24, 4, 1, 3, 6, 10, 7, 1, 3, 9, 4, 4, 21, 1, 19, 4, 14, 2, 1, 3, 6, 4, 13, 1, 8, 4, 3, 1, 8, 2, 5, 9, 1, 1, 1, 1, 1, 34, 9, 4, 14, 1, 3, 6, 10, 7, 1, 7, 2, 7, 7, 10, 4, 8, 1, 10, 8, 3, 2, 9, 12, 10, 19, 3, 1, 1, 40, 22, 2, 9, 16, 1, 18, 4, 15, 11, 1, 4, 18, 1, 3, 16, 9, 5, 8, 3, 1, 15, 10, 8, 17, 1, 

In [ ]:
data_encoded = list(itertools.chain(*data_encoded))

for i in range (len(data_encoded)):
  data_encoded[i] = data_encoded[i]-1

In [ ]:
data_encoded

[31,
 1,
 2,
 0,
 2,
 5,
 1,
 0,
 19,
 9,
 8,
 11,
 0,
 3,
 17,
 0,
 10,
 3,
 12,
 11,
 1,
 6,
 2,
 0,
 10,
 4,
 15,
 0,
 0,
 29,
 7,
 0,
 2,
 5,
 1,
 0,
 6,
 3,
 10,
 1,
 0,
 24,
 8,
 4,
 19,
 9,
 4,
 7,
 0,
 2,
 8,
 1,
 1,
 0,
 0,
 30,
 1,
 8,
 4,
 10,
 11,
 0,
 6,
 4,
 11,
 0,
 4,
 7,
 11,
 0,
 2,
 8,
 12,
 13,
 20,
 1,
 2,
 0,
 19,
 1,
 0,
 0,
 0,
 23,
 3,
 0,
 14,
 5,
 3,
 6,
 1,
 0,
 6,
 3,
 12,
 7,
 11,
 0,
 18,
 5,
 4,
 6,
 2,
 1,
 0,
 14,
 9,
 7,
 16,
 6,
 0,
 3,
 19,
 1,
 15,
 0,
 0,
 0,
 0,
 0,
 28,
 12,
 2,
 0,
 2,
 5,
 3,
 12,
 0,
 6,
 5,
 8,
 9,
 1,
 22,
 9,
 7,
 16,
 0,
 5,
 4,
 8,
 19,
 9,
 7,
 16,
 1,
 8,
 0,
 0,
 0,
 33,
 3,
 12,
 10,
 0,
 20,
 8,
 1,
 18,
 12,
 8,
 8,
 1,
 8,
 0,
 3,
 17,
 0,
 2,
 5,
 1,
 0,
 17,
 9,
 1,
 7,
 11,
 0,
 0,
 0,
 24,
 12,
 16,
 12,
 8,
 0,
 3,
 17,
 0,
 2,
 5,
 1,
 0,
 17,
 1,
 21,
 1,
 8,
 0,
 6,
 0,
 1,
 7,
 11,
 0,
 0,
 0,
 23,
 3,
 0,
 2,
 5,
 9,
 6,
 0,
 2,
 8,
 3,
 3,
 20,
 0,
 18,
 3,
 13,
 1,
 0,
 2,
 5,
 3,
 12,
 0,
 7,
 3,
 2,


In [ ]:
data = tf.data.Dataset.from_tensor_slices(data_encoded)
data = data.window(window_length, shift=n_steps, drop_remainder=True)
data = data.flat_map(lambda window : window.batch(window_length))
data = data.batch(1)
data = data.map(lambda windows : (windows[:, :-1], windows[:, 1:]))
data = data.map(lambda X_batch, Y_batch : (tf.one_hot(X_batch, depth=max_id), Y_batch))
data = data.prefetch(1)

# Preparing the model

Recurrenct models can be stateful or stateless. Stateful models learn faster and can find patterns in the data. The disadvantage of the stateful model is that input data must have fixed size. On the other hand, the stateless models learn slower than the stateful ones and cannot find the patterns but the input data can have any size. I'll use advantages of both types of models and first I'll train stateful model (which will learn fast and maybe will find any pattern in the training data) and in the next step I'll copy the weights from trained stateful model to stateless model, so user will be able to give any data to the input.

## Preparing the stateful model

### Implementing callback for reseting the model's state on the beginning of every epoch

In [ ]:
class ResetStatesCallback(Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

### Implementing callback for saving the model on the end of every epoch

In [ ]:
class SaveModelCallback(Callback):
  def on_epoch_end(self, epoch, logs):
    self.model.save(os.path.join(poem_generator_directory, stateful_rnn_file_name), save_format='h5')

### Implementing the model

In [ ]:
model = Sequential()
model.add(GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0.2, batch_input_shape=[1, None, max_id]))
model.add(GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0.2))
model.add(TimeDistributed(Dense(max_id, activation='softmax')))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (1, None, 128)            74496     
                                                                 
 gru_1 (GRU)                 (1, None, 128)            99072     
                                                                 
 time_distributed (TimeDist  (1, None, 64)             8256      
 ributed)                                                        
                                                                 
Total params: 181824 (710.25 KB)
Trainable params: 181824 (710.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

### Training

In [ ]:
model.fit(data, epochs=15, callbacks=[ResetStatesCallback(), SaveModelCallback()])

Epoch 1/15
5672/5672 [==============================] - 1625s 286ms/step - loss: 1.6367


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/15
5672/5672 [==============================] - 1622s 286ms/step - loss: 1.6344
Epoch 3/15
5672/5672 [==============================] - 1628s 287ms/step - loss: 1.6311
Epoch 4/15
5672/5672 [==============================] - 1634s 288ms/step - loss: 1.6301
Epoch 5/15
5672/5672 [==============================] - 1643s 290ms/step - loss: 1.6264
Epoch 6/15
5672/5672 [==============================] - 1650s 291ms/step - loss: 1.6242
Epoch 7/15
5672/5672 [==============================] - 1645s 290ms/step - loss: 1.6233
Epoch 8/15
5672/5672 [==============================] - 1635s 288ms/step - loss: 1.6211
Epoch 9/15
5672/5672 [==============================] - 1639s 289ms/step - loss: 1.6204
Epoch 10/15
5672/5672 [==============================] - 1631s 288ms/step - loss: 1.6178
Epoch 11/15
5672/5672 [==============================] - 1633s 288ms/step - loss: 1.6168
Epoch 12/15
5672/5672 [==============================] - 1618s 285ms/step - loss: 1.6148
Epoch 13/15
5672/5672 [======

### Saving the trained model

In [ ]:
model.save('/content/gdrive/MyDrive/poem_generator/model.h5', save_format='h5')

### Saving the tokenizer

In [ ]:
path = os.path.join(poem_generator_directory, tokenizer_file_name)
with open(path, 'wb') as pickle_file:
    pickle.dump(tokenizer, pickle_file)

## Preparing the stateless model

### Loading the stateful model from which the weights will be taken

In [ ]:
model = load_model('/content/gdrive/MyDrive/poem_generator/model.h5')

### Creating the stateless model

In [ ]:
stateless_model=tf.keras.models.Sequential()
stateless_model.add(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=[None, max_id]))
stateless_model.add(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
stateless_model.add(TimeDistributed(Dense(max_id, activation='softmax')))
stateless_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, None, 128)         74496     
                                                                 
 gru_3 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed_1 (TimeDi  (None, None, 64)          8256      
 stributed)                                                      
                                                                 
Total params: 181824 (710.25 KB)
Trainable params: 181824 (710.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Setting weights from stateful model to the stateless model

In [ ]:
stateless_model.set_weights(model.get_weights())

### Compiling the stateless model

In [ ]:
stateless_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

### Implementing callback for saving the stateless model on every epoch end

In [ ]:
class SaveStatelessModelCallback(Callback):
  def on_epoch_end(self, epoch, logs):
    self.model.save(os.path.join(poem_generator_directory, stateless_rnn_file_name), save_format='h5')

### Trainig the stateless model for a while

In [ ]:
stateless_model.fit(data, epochs=10, callbacks=[SaveStatelessModelCallback()])

Epoch 1/10
5672/5672 [==============================] - 2081s 367ms/step - loss: 1.6576
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5672/5672 [==============================] - 2042s 360ms/step - loss: 1.6548
Epoch 3/10
5672/5672 [==============================] - 1969s 347ms/step - loss: 1.6512
Epoch 4/10
5672/5672 [==============================] - 1956s 345ms/step - loss: 1.6487
Epoch 5/10
5672/5672 [==============================] - 1957s 345ms/step - loss: 1.6471
Epoch 6/10
5672/5672 [==============================] - 1955s 345ms/step - loss: 1.6441
Epoch 7/10
5672/5672 [==============================] - 1935s 341ms/step - loss: 1.6421
Epoch 8/10
5672/5672 [==============================] - 1929s 340ms/step - loss: 1.6416
Epoch 9/10
5672/5672 [==============================] - 2083s 367ms/step - loss: 1.6390
Epoch 10/10
4624/5672 [=======================>......] - ETA: 6:24 - loss: 1.6346

There is no need for saving this model because model was saved after every epoch

# Loading the models

In [ ]:
def load_tokenizer(filename):
    with open(filename, 'rb') as pickle_file:
        return pickle.load(pickle_file)

In [ ]:
stateless_model = load_model(os.path.join(poem_generator_directory, stateless_rnn_file_name))
tokenizer = load_tokenizer(os.path.join(poem_generator_directory, tokenizer_file_name))

# Generating the poem

Function encodes the given text

In [ ]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts)) - 1
  return tf.one_hot(X, max_id)

Function predicts and returns next character in the given sequence

In [ ]:
def next_char(model, text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

Function generates poem starting on the given sequence, length and temperature. There is also possibility to use another model in the future.

In [ ]:
def generate_poem(model, start_string='A', len=50, temperature=1):
    text = start_string
    for _ in range(len):
      text += next_char(model=model, text=text, temperature=temperature)
    text =  re.sub(r'\s{2,}', '\n', text)
    return text

Enjoy the poems 😀

In [ ]:
poem = generate_poem(stateless_model, start_string='You ', len=500)
print(poem)

1/1 [==============================] - 0s 133ms/step
You a mettle in the cold
Taker the blampering of his gasty scill of the sun life
And sparrieve thus of my souls
Indeep the find s trieble drew
In the saught but he immand standring
Put you fit we wavrs
and the wead some
exering to keepy of Live thou not what an this look
exilere may croff
And the garmories breaded brossing in tears he still long
Yet
I walkbing
In a year of her puriiant of gray
Not take the chuibity
her groweninor lights
Momening snow of throy and quaith o
